### 1.1. Escrapear datos

Descripcion: Utilizamos los datos reflejados en la web fuente para la creación de dos DataFrame.

#### 1.1.1. Web pagina Inicial Vinos "Sin Alcohol".
Tomamos la lista de 63 tipos de vinos y obtuvimos los principales datos sobre cada uno:
- Nombre
- Variedad
- Precio

#### 1.1.2. Webs secuntarias, entramos a la url por cada vino disponible en la pagina inicial.
Obtuvimos los datos unicos sobre cada uno:
- Volumen
- Variedad
- Graduación
- Productor
- Etiqueta

ESCRAPEO DE DATOS

Fuente: https://www.thebluedolphinstore.com/

### 1.1.1
Paso 1: instalamos selenium e importamos librerias

In [328]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [500]:
import requests as req

from bs4 import BeautifulSoup as bs

import pandas as pd

In [501]:
url = 'https://www.thebluedolphinstore.com/categoria-producto/vinos-sin-alcohol/'

In [502]:
html=req.get(url).text

sopa=bs(html, 'html.parser')


In [503]:
from selenium import webdriver   # driver de selenium

from selenium.webdriver.chrome.options import Options   # opciones del driver

from selenium.webdriver.common.by import By # By es para buscar por tag, clase, id...

from selenium.webdriver.common.keys import Keys  # para usar teclado

In [504]:
PATH = 'driver/chromedriver'

In [505]:
# opciones del driver
opciones=Options()

# quita la bandera de ser robot
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)


# guardar las cookies
opciones.add_argument('user-data-dir=cookies')    # mantiene las coockies

In [517]:
driver = webdriver.Chrome()

driver.get(url) 

 Paso 2: comenzamos la busqueda de datos

In [574]:
productos = driver.find_elements(By.CLASS_NAME, "product-wrap")

len(productos)

63

In [569]:
productos[0].text.split('\n')[1].split('–')

['Carl Jung Rosé ', ' Vino Rosado Sin Alcohol']

In [570]:
nombre_variedad = productos[2].find_element(By.TAG_NAME, 'h3').text.split(' – ')

nombre_variedad

['Carl Jung Cabernet Sauvignon', 'Vino Tinto Sin Alcohol']

In [565]:
nombre = nombre_variedad[0]

nombre

'Carl Jung Cabernet Sauvignon'

In [566]:
variedad = nombre_variedad[1]

variedad

'Vino Tinto Sin Alcohol'

In [531]:
precio = productos[0].find_element(By.CSS_SELECTOR,'#ajax-content-wrap > div.container-wrap > div > div.row > div.post-area.col.span_9.col_last > ul > li.minimal.product.type-product.post-28958.status-publish.first.instock.product_cat-destacados.product_cat-vino-rosado-sin-alcohol.product_cat-vinos-sin-alcohol.product_tag-vegana-o.has-post-thumbnail.taxable.shipping-taxable.purchasable.product-type-simple.yith-wcbm-product-has-badges.hover-bound > div.product-wrap > div > div > span > span > bdi').text

precio

'6,86€'

In [532]:
precio = float(precio.replace('€', '').replace(',', '.'))

precio

6.86

Paso 3: Obtenemos la data y creamos el DataFrame

In [581]:
data = []

for p in productos[0:]:
    
    nombre_variedad = p.find_element(By.TAG_NAME, 'h3').text.split(' – ')
    nombre_variedad = productos[2].find_element(By.TAG_NAME, 'h3').text.split(' – ')
    nombre = nombre_variedad[0] 
    
    #precio = p.find_element(By.CSS_SELECTOR,'#ajax-content-wrap > div.container-wrap > div > div.row > div.post-area.col.span_9.col_last > ul > li.minimal.product.type-product.post-28958.status-publish.first.instock.product_cat-destacados.product_cat-vino-rosado-sin-alcohol.product_cat-vinos-sin-alcohol.product_tag-vegana-o.has-post-thumbnail.taxable.shipping-taxable.purchasable.product-type-simple.yith-wcbm-product-has-badges.hover-bound > div.product-wrap > div > div > span > span > bdi').text
    
    
    #precio = float(precio.replace('€', '').replace('\n', '.'))
    
    
    dictio = {'nombre': nombre, 'variedad' : nombre_variedad[1], 'precio': precio}
    
    data.append(dictio)

In [582]:
data

[{'nombre': 'Carl Jung Cabernet Sauvignon',
  'variedad': 'Vino Tinto Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Carl Jung Cabernet Sauvignon',
  'variedad': 'Vino Tinto Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Carl Jung Cabernet Sauvignon',
  'variedad': 'Vino Tinto Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Carl Jung Cabernet Sauvignon',
  'variedad': 'Vino Tinto Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Carl Jung Cabernet Sauvignon',
  'variedad': 'Vino Tinto Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Carl Jung Cabernet Sauvignon',
  'variedad': 'Vino Tinto Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Carl Jung Cabernet Sauvignon',
  'variedad': 'Vino Tinto Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Carl Jung Cabernet Sauvignon',
  'variedad': 'Vino Tinto Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Carl Jung Cabernet Sauvignon',
  'variedad': 'Vino Tinto Sin Alcohol',
  'precio': 6.86},
 {'nombre': 'Carl Jung Cabernet Sauvignon',
  'variedad': 'Vino Tinto Sin

In [584]:
## primer DataFrame listo:

listado = pd.DataFrame(data)

listado.head()

,nombre,variedad,precio
0,Carl Jung Cabernet Sauvignon,Vino Tinto Sin Alcohol,6.86
1,Carl Jung Cabernet Sauvignon,Vino Tinto Sin Alcohol,6.86
2,Carl Jung Cabernet Sauvignon,Vino Tinto Sin Alcohol,6.86
3,Carl Jung Cabernet Sauvignon,Vino Tinto Sin Alcohol,6.86
4,Carl Jung Cabernet Sauvignon,Vino Tinto Sin Alcohol,6.86


### 1.1.2

Obtenemos datos especificios sobre cada producto de la pagina inicial.
Está dividido en dos partes:

Parte I:

Paso 1: Conseguir las URL de cada producto

In [585]:
lista_urls = []

In [586]:
lista_urls = []
for div in sopa.find_all('div', {'class':'product-wrap'}): 
    url = div.find('a').attrs['href']
    lista_urls.append(url)

Paso 2. Defino las características que describen cada botella de vino y posteriormente lo aplico a cada URL

In [587]:
url_producto_indv = "https://www.thebluedolphinstore.com/producto/carl-jung-rose-vino-rosado-sin-alcohol/"

In [588]:
driver = webdriver.Chrome()   # abre una ventana de chrome

driver.get(url_producto_indv)   # entra en la pagina

In [589]:
producto_indv = driver.find_element(By.CLASS_NAME, "woocommerce-product-details__short-description").text.split('\n')

producto_indv

['Vino rosado sin alcohol',
 'Carl Jung Rosé es un delicioso vino rosado sin alcohol que tiene hasta 1/3 menos calorías que un vino rosado normal. Por tanto permite disfrutar de forma más saludable y si estás a dieta, sin perder nada de sabor.',
 'Este vino rosado sin alcohol de Carl Jung tiene todas las ventajas de un rosado típico pero nada de alcohol. Además, esta marca produce vinos sin alcohol veganos.',
 'Notas de Cata',
 'De color rosa sutil, ligeramente brillante, finamente afrutado y refrescante. Acidez armoniosa, taninos sutiles, frutal con recuerdos a frambuesas y fresas.',
 'Volumen: 75 cl.',
 'Variedad:',
 'Graduación: <0,5 %',
 'Productor: Carl Jung (Alemania)']

In [590]:
notas_de_cata = producto_indv[4]
notas_de_cata

'De color rosa sutil, ligeramente brillante, finamente afrutado y refrescante. Acidez armoniosa, taninos sutiles, frutal con recuerdos a frambuesas y fresas.'

In [591]:
for element in producto_indv:
    if element.startswith('Volumen'):
        volumen = int(element.replace('Volumen: ', '').replace('<.','').replace('cl.',''))
    if element.startswith('Variedad'):
        variedad = element.replace('Variedad: ', '')
    if element.startswith('Graduación'): 
        graduacion = element.replace('Graduación: ', '')
    if element.startswith('Productor'): 
        productor = element.replace('Productor: ', '')
    if element.startswith('Productor'):
        pais = element.split('(')[1].replace(')','')
    if element in producto_indv[4]:
        notas_de_cata = notas_de_cata

In [592]:
volumen

75

In [593]:
variedad

'Variedad:'

In [594]:
graduacion

'<0,5 %'

In [595]:
productor

'Carl Jung (Alemania)'

In [596]:
pais

'Alemania'

In [597]:
etiqueta = driver.find_element(By.CLASS_NAME, "tagged_as").text

etiqueta

'Etiqueta: Vegano'

In [598]:
etiqueta = etiqueta.replace('Etiqueta: ', '')
        
etiqueta

'Vegano'

In [599]:
## Tuve que instalar el paquete de selenium porque me daba esta excepcion en la etiqueta
from selenium.common.exceptions import NoSuchElementException

Paso 3: Creo la data2 con cada valor unico y obtenemos el siguiente DataFrame
Este paso me ha dado muchos errores:
1. He tenido que incluir muchos valores de .replace() porque no en todas las url estan definidos de la misma forma, y por ello me cortaba el bucle una y otra vez.
2. Cada pagina tiene que abrir, buscar y cerrarse, por eso agrego los driver.get() y driver.close()
3. Etiqueta me daba muchos errores agregamos la excepcion de NoSuchElementException y le pedimos que cuando esto ocurra nos arroje el valor de No Disponible.
4. Paciencia.

In [600]:
data2 = []

for u in lista_urls:
    driver = webdriver.Chrome()
    driver.get(u) 
   
    producto_indv = driver.find_element(By.CLASS_NAME, "woocommerce-product-details__short-description").text.split('\n')
    
    for element in producto_indv:
        if element.startswith('Volumen'):
            volumen = int(element.replace('Volumen: ', '').replace('75 .','').replace('.','').replace('cl','').replace('cl.',''))
        if element.startswith('Variedad'):
            variedad = element.replace('Variedad: ', '')
        if element.startswith('Graduación'): 
            graduacion = element.replace('Graduación: ', '')
        if element.startswith('Productor'): 
            productor = element.replace('Productor: ', '')
        if element.startswith('Productor'):
            pais = element.split('(')[1].replace(')','')
        if element in producto_indv[4]:
            notas_de_cata = notas_de_cata
    try:
        etiqueta = driver.find_element(By.CLASS_NAME, "tagged_as").text.replace('Etiqueta: ', '')
        etiqueta = etiqueta.replace('Etiqueta: ', '').replace('Etiquetas: ' ,'')
    except NoSuchElementException:
        etiqueta = "No Disponible"

    
    dictio2 = {'volumen' : volumen, 'graduacion': graduacion, 'productor':productor, 'pais': pais, 'etiqueta': etiqueta, 'notas de cata': notas_de_cata,'url': url}
    
    data2.append(dictio2)
    
    driver.close()
    

In [601]:
data2

[{'volumen': 75,
  'graduacion': '<0,5 %',
  'productor': 'Carl Jung (Alemania)',
  'pais': 'Alemania',
  'etiqueta': 'Vegano',
  'notas de cata': 'De color rosa sutil, ligeramente brillante, finamente afrutado y refrescante. Acidez armoniosa, taninos sutiles, frutal con recuerdos a frambuesas y fresas.',
  'url': 'https://www.thebluedolphinstore.com/producto/vino-sin-alcohol-zero-zero-blanco/'},
 {'volumen': 75,
  'graduacion': '0,5%',
  'productor': 'Carl Jung (Alemania)',
  'pais': 'Alemania',
  'etiqueta': 'Syrah',
  'notas de cata': 'De color rosa sutil, ligeramente brillante, finamente afrutado y refrescante. Acidez armoniosa, taninos sutiles, frutal con recuerdos a frambuesas y fresas.',
  'url': 'https://www.thebluedolphinstore.com/producto/vino-sin-alcohol-zero-zero-blanco/'},
 {'volumen': 75,
  'graduacion': '<0,5 %',
  'productor': 'Carl Jung (Alemania)',
  'pais': 'Alemania',
  'etiqueta': 'Cabernet Sauvignon, Vegano',
  'notas de cata': 'De color rosa sutil, ligeramente br

In [602]:
listado_2 = pd.DataFrame(data2)

listado_2.head()

,volumen,graduacion,productor,pais,etiqueta,notas de cata,url
0,75,"<0,5 %",Carl Jung (Alemania),Alemania,Vegano,"De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...
1,75,"0,5%",Carl Jung (Alemania),Alemania,Syrah,"De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...
2,75,"<0,5 %",Carl Jung (Alemania),Alemania,"Cabernet Sauvignon, Vegano","De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...
3,75,"<0,5 %",Carl Jung (Alemania),Alemania,"Merlot, Vegano","De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...
4,75,"0,5%",Carl Jung (Alemania),Alemania,No Disponible,"De color rosa sutil, ligeramente brillante, fi...",https://www.thebluedolphinstore.com/producto/v...


Parte 2:  

Paso 4: Escrapeo de informacion nutricional 

In [603]:
ulr_nutricional = 'https://www.thebluedolphinstore.com/producto/carl-jung-rose-vino-rosado-sin-alcohol/'

In [604]:
driver = webdriver.Chrome()   # abre una ventana de chrome

driver.get(ulr_nutricional)   # entra en la pagina

In [605]:
descripcion_nutricional = driver.find_element(By.CLASS_NAME, "nectar-prod-wrap").text.split('\n')

descripcion_nutricional

['🔍',
 'NUEV@',
 'Carl Jung Rosé – Vino Rosado Sin Alcohol',
 '6,86€ IVA inc.',
 'COMPRAR',
 'Vino rosado sin alcohol',
 'Carl Jung Rosé es un delicioso vino rosado sin alcohol que tiene hasta 1/3 menos calorías que un vino rosado normal. Por tanto permite disfrutar de forma más saludable y si estás a dieta, sin perder nada de sabor.',
 'Este vino rosado sin alcohol de Carl Jung tiene todas las ventajas de un rosado típico pero nada de alcohol. Además, esta marca produce vinos sin alcohol veganos.',
 'Notas de Cata',
 'De color rosa sutil, ligeramente brillante, finamente afrutado y refrescante. Acidez armoniosa, taninos sutiles, frutal con recuerdos a frambuesas y fresas.',
 'Volumen: 75 cl.',
 'Variedad:',
 'Graduación: <0,5 %',
 'Productor: Carl Jung (Alemania)',
 'Categorías: Destacados, Vino Rosado Sin Alcohol, Vinos Sin Alcohol',
 'Etiqueta: Vegano',
 'Descripción',
 'Este vino rosado sin alcohol es ideal para todos los momentos, además es vegano. Elaborado por la bodega alemana 

In [606]:
calorias = descripcion_nutricional[23]
calorias

'Calorías:  19,4 Kcal. ( 81,3 Kj.)'

In [607]:
for element in descripcion_nutricional:
    if element.startswith('Calorías'):
        calorias = element.replace('Calorías: ', '').replace('Calorias:','').replace('Kcal.','').replace('Kcal','').replace(' ','').replace('kJ','').replace('(7','').replace('.k','').replace('(8','').replace('(9','')
        calorias = calorias[0:4]
    if element.startswith('Proteínas'):
        proteinas = element.replace('Proteínas: ', '').replace('Proteinas: ', '').replace('g. ', '').replace('g ', '').replace('.', '').replace('<1', '0,1').replace('0,o2', '0,2').replace('<0,01', '0,1').replace('0 g', '0,00')
        proteinas = proteinas[:-1].strip()
    if element.startswith('Grasa'): 
        grasa = element.replace('Grasa: ', '').replace('g.: ', '').replace('g ', '').replace('.','').replace('Grasa: ', '').replace(' ','').replace('Grasas:0g ','0').replace('Grasas:0','0').replace('Grasas:0','0').replace('Grasas:<1','1').replace('<0,50','0,5').replace('<0,05','0,5').replace('<0,01','0,1').replace('0g','0,00')
        grasa = grasa[:-1].strip()
    if element.startswith('Hidratos de Carbono'): 
        hidratos_carbono = element.replace('Hidratos de Carbono: ', '').replace('g. ', '').replace('g ', '').replace('.','')
        hidratos_carbono = hidratos_carbono[:-1].strip()
    if element.startswith('Azúcares totales'):
        azucar = element.replace('Azúcares totales: ', '').replace('Azucares totales: ', '').replace('g. ', '').replace('g: ', '').replace('.','')
        azucar = azucar[:-1].strip()
    if element.startswith('Sal'): 
        sal = element.replace('Sal: ', '').replace("']", '').replace('.','').replace('g ', '')
        sal[:-1].strip()

In [608]:
calorias

'19,4'

In [609]:
proteinas

'0,0'

In [610]:
grasa

'0,0'

In [611]:
 hidratos_carbono

'4,2'

In [612]:
azucar

'3,5'

In [613]:
sal

'<0,1 g'

In [614]:
data3 = []

for u in lista_urls:
    driver = webdriver.Chrome()
    driver.get(u) 
   
    descripcion_nutricional = driver.find_element(By.CLASS_NAME, "nectar-prod-wrap").text.split('\n')
    
    for element in descripcion_nutricional:
        if element.startswith('Calorías'):
            calorias = element.replace('Calorías: ', '').replace('Calorias:','').replace('Kcal.','').replace('Kcal','').replace(' ','').replace('kJ','').replace('(7','').replace('.k','').replace('(8','').replace('(9','')
            calorias = calorias[0:4]
        if element.startswith('Proteínas'):
            proteinas = element.replace('Proteínas: ', '').replace('Proteinas: ', '').replace('g. ', '').replace('g ', '').replace('.', '').replace('<1', '0,1').replace('0,o2', '0,2').replace('<0,01', '0,1').replace('0 g', '0,00')
            proteinas = proteinas[:-1].strip()
        if element.startswith('Grasa'): 
            grasa = element.replace('Grasa: ', '').replace('g.: ', '').replace('g ', '').replace('.','').replace('Grasa: ', '').replace(' ','').replace('Grasas:0g ','0').replace('Grasas:0','0').replace('Grasas:0','0').replace('Grasas:<1','1').replace('<0,50','0,5').replace('<0,05','0,5').replace('<0,01','0,1').replace('0g','0,00')
            grasa = grasa[:-1].strip()
        if element.startswith('Hidratos de Carbono'): 
            hidratos_carbono = element.replace('Hidratos de Carbono: ', '').replace('g. ', '').replace('g ', '').replace('.','')
            hidratos_carbono = hidratos_carbono[:-1].strip()
        if element.startswith('Azúcares totales'):
            azucar = element.replace('Azúcares totales: ', '').replace('Azucares totales: ', '').replace('g. ', '').replace('g: ', '').replace('.','')
            azucar = azucar[:-1].strip()
        if element.startswith('Sal'): 
            sal = element.replace('Sal: ', '').replace("']", '').replace('.','').replace('g ', '')
            sal[:-1].strip()

    
    dictio3 = {'calorias' : calorias, 'proteinas': proteinas, 'grasa': grasa, 'hidratos de carbono': hidratos_carbono, 'azucar': azucar, 'sal': sal}
    
    data3.append(dictio3)
    
    driver.close()

In [615]:
data3

[{'calorias': '19,4',
  'proteinas': '0,0',
  'grasa': '0,0',
  'hidratos de carbono': '4,2',
  'azucar': '3,5',
  'sal': '<0,1 g'},
 {'calorias': '79.(',
  'proteinas': '0,5',
  'grasa': '0,0',
  'hidratos de carbono': '4,8',
  'azucar': '4,2',
  'sal': '<0,1 g'},
 {'calorias': '19,4',
  'proteinas': '0,0',
  'grasa': '0,0',
  'hidratos de carbono': '4,2',
  'azucar': '3,5',
  'sal': '<0,1 g'},
 {'calorias': '22,9',
  'proteinas': '0,0',
  'grasa': '0,0',
  'hidratos de carbono': '4,7',
  'azucar': '4,0',
  'sal': '<0,1 g'},
 {'calorias': '95.(',
  'proteinas': '0,5',
  'grasa': '0,3',
  'hidratos de carbono': '5,6',
  'azucar': '4,6',
  'sal': '<0,1 g'},
 {'calorias': '95.(',
  'proteinas': '0,5',
  'grasa': '0,0',
  'hidratos de carbono': '5,6',
  'azucar': '4,6',
  'sal': '<0,1 g'},
 {'calorias': '20,1',
  'proteinas': '0,0',
  'grasa': '0,0',
  'hidratos de carbono': '4,4,',
  'azucar': '4,0',
  'sal': '<0,1 g'},
 {'calorias': '185,',
  'proteinas': '0,0',
  'grasa': '0,0',
  'hid

In [616]:
listado_3 = pd.DataFrame(data3)

listado_3.head()

,calorias,proteinas,grasa,hidratos de carbono,azucar,sal
0,"19,4","0,0","0,0","4,2","3,5","<0,1 g"
1,79.(,"0,5","0,0","4,8","4,2","<0,1 g"
2,"19,4","0,0","0,0","4,2","3,5","<0,1 g"
3,"22,9","0,0","0,0","4,7","4,0","<0,1 g"
4,95.(,"0,5","0,3","5,6","4,6","<0,1 g"


In [617]:
listado_3.shape

(63, 6)

Paso 5: Exportar a CSV cada DataFrame

In [618]:
listado.to_csv ('listado.csv' , index = False)
listado_2.to_csv ('listado2.csv' , index = False)
listado_3.to_csv ('listado3.csv' , index = False)